In [1]:
import re
from pathlib import Path
import pandas as pd
import spacy
from segram import Corpus, Story

# Read dataset of articles coveriing the war in Syria
data = pd.read_csv(Path(".")/"data"/"FA-KES-Dataset.csv", encoding="latin")

# Sanitize beginnings of content strings to get rid of publication dates
# and other metadata. This is done using a few regular expressions
#
# Get rid of publication dates
pattern1 = r"^(" \
    r"\s*([A-Z]\w+)?\s*\d{1,2}\s*([A-Z]\w+)?\s*\d{2,4}\s*" \
    r"|" \
    r"\d{1,2}[-\.]\d{1,2}[-\.]\d{2,4}\s*" \
    r")" \
    r"(\s*at\s*\d+:?\d+)?"
# Get rid of update dates
pattern2 = r"^\s*(\(updated\s*[\w\d]*\s*\))\s*"
# Get rid of get url shout outs
pattern3 = r"^\s*get\s*short\s*url\s*([\d\w]+\s*)*"

# Sanitize the content column
data["article_content"] = data["article_content"] \
    .str.replace(pattern1, r"", regex=True) \
    .str.replace(pattern2, r"", regex=True, flags=re.IGNORECASE) \
    .str.replace(pattern3, r"", regex=True, flags=re.IGNORECASE) \
    .str.strip()

In [2]:
nlp = spacy.load("en_core_web_trf")
nlp.add_pipe("segram", config={
    "vectors": "en_core_web_lg"
})
nlp.add_pipe("segram_coref")

In [3]:
titles = Corpus.from_texts(nlp, *data["article_title"], progress=True)

0it [00:00, ?it/s]

In [4]:
fpath = Path(".")/"data"/"titles.segram"
titles.to_disk(fpath, vocab=False, nlp=False)

In [14]:
fpath.absolute()

PosixPath('/home/sztal/Coding/python/segram/examples/data/titles.segram')

In [12]:
Corpus.from_disk(fpath, vocab=nlp.vocab)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/sztal/miniconda3/envs/segram/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_32339/1391504927.py", line 1, in <module>
    Corpus.from_disk(fpath)
  File "/home/sztal/Coding/python/segram/segram/nlp/corpus.py", line 314, in from_disk
  File "/home/sztal/Coding/python/segram/segram/nlp/corpus.py", line 271, in from_data
    data["vocab"] = vocab
                          
KeyError: 'vocab'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/sztal/miniconda3/envs/segram/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/sztal/miniconda3/envs/segram/lib/python3.11/site-packages/IPython/core/ultratb.py", lin

In [8]:
D = Corpus.from_disk("test")